## UM-Bridge Integration for Predator-Prey Model

### The Model

We define the predator-prey model in the UM-Bridge framework. By executing the following code lines, a server starts attending for client requests. Input parameters are time point $t$, initial conditions for prey and predator and the parameters $\theta_{1}, \theta_{2}, \theta_{12}, \theta_{21}$ determing the dynamical system.

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp # for solving the ode
import matplotlib.pyplot as plt
import nest_asyncio
nest_asyncio.apply()
import math
import scipy.integrate as integrate
import scipy.special as special
import umbridge

class ODEModel(umbridge.Model):
    def __init__(self):
        super().__init__("forward")
        self.A=[]
    
    def get_input_sizes(self, config): 
        return [7]  # for the input t, u0_prey, u0_predator, theta1, theta2, theta12, theta21
        
    def get_output_sizes(self, config):
        return [2]  # for the output u1 and u2 (prey and predator)

    def __call__(self, parameters, config):
        T = parameters[0][0]
        u0_prey = parameters[0][1] 
        u0_predator = parameters[0][2]
        theta1 = parameters[0][3]
        theta2 = parameters[0][4]
        theta12 = parameters[0][5]
        theta21 = parameters[0][6]
        u0_bar = [u0_prey, u0_predator ]
        sol = solve_ivp(self.lotka_volterra, [0, T], u0_bar, args=(theta1, theta2, theta12, theta21, config), method='RK45', t_eval=[T])
        result = [float(sol.y[0, -1]), float(sol.y[1, -1])]
        return [result]

    #solve the Lotka-Volterra model and consider u1 as prey, u2 as predator
    def lotka_volterra(self, t, u, theta1, theta2, theta12, theta21, config):
        u1, u2 = u
        du1dt = theta1 * u1 - theta12 * u1 * u2
        du2dt = theta21 * u1 * u2 - theta2 * u2
        return [du1dt, du2dt]


    def supports_evaluate(self):
        return True

ode_model = ODEModel()
umbridge.serve_models([ode_model], 4242) # starting the server

======== Running on http://0.0.0.0:4244 ========
(Press CTRL+C to quit)
